In [25]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time
import numpy as np
import pandas as pd
import tensorflow as tf

In [26]:
tf.logging.set_verbosity(tf.logging.INFO)
# Set to INFO for tracking training, default is WARN 

print("Using TensorFlow version %s" % (tf.__version__))



Using TensorFlow version 1.5.0


In [3]:
%%time
print('loading files ...')
myfolder = 'C:/Users/bridge/RecSys/'

prior = pd.read_csv(myfolder + 'order_products__prior.csv', dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.uint8, 'add_to_cart_order': np.uint8})

train_orders = pd.read_csv(myfolder + 'order_products__train.csv', dtype={'order_id': np.uint32,
           'product_id': np.uint16, 'reordered': np.int8, 'add_to_cart_order': np.uint8 })

orders = pd.read_csv(myfolder + 'orders.csv', dtype={'order_hour_of_day': np.uint8,
           'order_number': np.uint8, 'order_id': np.uint32, 'user_id': np.uint32,
           'order_dow': np.uint8, 'days_since_prior_order': np.float16})
orders.eval_set = orders.eval_set.replace({'prior': 0, 'train': 1, 'test':2}).astype(np.uint8)
orders.days_since_prior_order = orders.days_since_prior_order.fillna(30).astype(np.uint8)

products = pd.read_csv(myfolder + 'products.csv', dtype={'product_id': np.uint16,
            'aisle_id': np.uint8, 'department_id': np.uint8},
             usecols=['product_id', 'aisle_id', 'department_id'])
print('done loading')

loading files ...
done loading
Wall time: 16.8 s


In [4]:
print("merging orders and prior...")
df = orders.merge(prior, how = 'inner', on = 'order_id')
# print("merging orders and train")

merging orders and prior...


In [20]:
print(df.user_id.nunique())
print(df.product_id.nunique())

206209
49677


In [5]:
# total orders per user
df01 = orders[orders['eval_set'] == 0].groupby(['user_id'])['order_number'].max().to_frame('user_orders')
df01.head()

,user_orders
user_id,
1,10
2,14
3,12
4,5
5,4


In [59]:
# df01.describe
len(df01[df01['user_orders'] > 2])

206209

In [31]:
bigusers = df01[df01['user_orders'] > 25]
smallusers = df01[df01['user_orders'] <= 25]

In [32]:
bigusers.head()

,user_orders
user_id,
17,40
21,33
27,81
36,37
50,67


In [15]:
len(df01[df01['user_orders'] >= 25])

38025

In [11]:
len(bigusers.user_orders)

35965

In [35]:
bigusers.reset_index(level=0, inplace=True)

In [36]:
bigusers.head()

,user_id,user_orders
0,17,40
1,21,33
2,27,81
3,36,37
4,50,67


In [37]:
bigusers2 = bigusers.merge(df)

In [39]:
bigusers2.user_id.nunique()

35965

In [40]:
smallusers.reset_index(level=0, inplace=True)

In [42]:
smallusers2 = smallusers.merge(df)

In [43]:
smallusers2.user_id.nunique()

170244

In [44]:
bigusers3 = bigusers2.merge(products)

In [58]:
bigusers3.user_id.nunique()

35965

In [45]:
smallusers3 = smallusers2.merge(products)

In [48]:
train_orders = train_orders.merge(orders, left_on = 'order_id', right_on = 'order_id', how = 'inner')

In [49]:
train_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,1,4,4,10,9
1,1,11109,2,1,112108,1,4,4,10,9
2,1,10246,3,0,112108,1,4,4,10,9
3,1,49683,4,0,112108,1,4,4,10,9
4,1,43633,5,1,112108,1,4,4,10,9


In [50]:
trainorders = train_orders.merge(products)

In [51]:
testbig = trainorders.loc[trainorders['user_id'].isin(bigusers3['user_id'])]

In [52]:
testbig.user_id.nunique()

22932

In [53]:
trainorders.user_id.nunique()

131209

In [55]:
testsmall = trainorders.loc[trainorders['user_id'].isin(smallusers3['user_id'])]

In [56]:
testsmall.user_id.nunique()

108277

In [57]:
bigusers3.to_csv('bigtrain.csv', encoding='utf-8', index=False)

In [59]:
smallusers3.to_csv('smalltrain.csv', encoding='utf-8', index=False)

In [60]:
testbig.to_csv('bigtest.csv', encoding='utf-8', index=False)

In [61]:
testsmall.to_csv('smalltest.csv', encoding='utf-8', index=False)

In [41]:
len(smallusers.user_orders)

170244

In [8]:
%%time
# takes a few min
dfbig = df.groupby('order_id').filter(lambda x: len(x) > 25)

In [12]:
dfbig.user_id.nunique()

40187

In [20]:
df[''] = df.groupby(['order_id']).size()

In [22]:
big01.shape

(3214874,)

In [69]:
# dfsmall = df.groupby('order_id').filter(lambda x: len(x) <= 25)

In [71]:
dfbig.shape

(4567900, 10)

In [75]:
%%time
train_orders = train_orders.merge(orders, left_on = 'order_id', right_on = 'order_id', how = 'inner')

Wall time: 1.29 s


In [77]:
train_orders.head()

,order_id,product_id,add_to_cart_order,reordered,user_id_x,user_id_y,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,112108,1,4,4,10,9
1,1,11109,2,1,112108,112108,1,4,4,10,9
2,1,10246,3,0,112108,112108,1,4,4,10,9
3,1,49683,4,0,112108,112108,1,4,4,10,9
4,1,43633,5,1,112108,112108,1,4,4,10,9


In [78]:
del train_orders['user_id_y']

In [79]:
train_orders.columns = train_orders.columns.str.replace('user_id_x','user_id')

In [80]:
traindf = train_orders
traindf.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,1,4,4,10,9
1,1,11109,2,1,112108,1,4,4,10,9
2,1,10246,3,0,112108,1,4,4,10,9
3,1,49683,4,0,112108,1,4,4,10,9
4,1,43633,5,1,112108,1,4,4,10,9


In [81]:
traindf.shape

(1384617, 10)

In [82]:
del train_orders

In [83]:
dftrain = traindf.loc[traindf['user_id'].isin(dfbig['user_id'])]

In [84]:
dftrain.shape

(464397, 10)

In [85]:
dropped_dftrain = traindf.loc[~traindf['user_id'].isin(dfbig['user_id'])]

In [86]:
dropped_dftrain.shape

(920220, 10)

In [89]:
dropped_dftrain.to_csv('dropped_dftrain.csv', encoding='utf-8', index=False)

In [90]:
dftrain.to_csv('dftrain.csv', encoding='utf-8', index=False)

In [91]:
xyz = pd.read_csv(myfolder + 'dftrain.csv')
print(xyz.shape)
xyz.head()

(464397, 10)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,98,8859,1,1,56463,1,41,3,8,14
1,98,19731,2,1,56463,1,41,3,8,14
2,98,43654,3,1,56463,1,41,3,8,14
3,98,13176,4,1,56463,1,41,3,8,14
4,98,4357,5,1,56463,1,41,3,8,14


In [92]:
dfbig.to_csv('dfbig.csv', encoding='utf-8', index=False)

In [14]:
#takes more than 8-10 min
dfsmall = df.groupby('order_id').filter(lambda x: len(x) <= 25)

In [18]:
dfsmall.user_id.nunique()

205790

In [19]:
dfsmall.shape

(27866589, 10)

In [94]:
dfsmall.to_csv('dfsmall.csv', encoding='utf-8', index=False)

In [95]:
train = dfbig.merge(products)
train.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,aisle_id,department_id
0,1718559,2,0,9,2,9,8,32792,1,1,23,19
1,2757999,4297,0,1,5,16,30,32792,10,0,23,19
2,653008,4783,0,1,1,11,30,32792,3,0,23,19
3,833607,6098,0,7,4,14,29,32792,18,0,23,19
4,1704427,10025,0,4,2,12,8,32792,29,0,23,19


In [96]:
train.to_csv('train00.csv', encoding='utf-8', index=False)

In [97]:
test = dftrain.merge(products)
test.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,aisle_id,department_id
0,98,8859,1,1,56463,1,41,3,8,14,115,7
1,1032,8859,4,1,164808,1,18,1,15,30,115,7
2,12847,8859,19,1,37602,1,5,6,20,30,115,7
3,41351,8859,2,1,46479,1,40,2,16,30,115,7
4,63515,8859,12,1,194154,1,26,3,16,12,115,7


In [98]:
test.to_csv('test00.csv', encoding='utf-8', index=False)

In [2]:
myfolder = 'C:/Users/bridge/RecSys/'
zxc = pd.read_csv(myfolder + 'train00.csv')
asd = pd.read_csv(myfolder + 'test00.csv')

In [3]:
zxc.user_id.nunique()

40187

In [4]:
asd.user_id.nunique()

25608